In [1]:
import flask

import numpy as np
import pandas as pd
import pickle
from collections import defaultdict

from sklearn.neighbors import NearestNeighbors


from fuzzywuzzy import process
from fuzzywuzzy import fuzz



In [2]:
with open('./data/df_merged_cleaned.pkl', 'rb') as picklefile:
    df_merged = pickle.load(picklefile)

with open('./data/topic_names.pkl', 'rb') as picklefile:
    topic_names = pickle.load(picklefile)

with open('./data/lda_model.pkl', 'rb') as picklefile:
    lda_mod = pickle.load(picklefile)

with open('./data/vectorizer.pkl', 'rb') as picklefile:
    vect_mod = pickle.load(picklefile)

with open('./data/lda_model_data.pkl', 'rb') as picklefile:
    lda_data = pickle.load(picklefile)

cleaned_talks = df_merged['text']

titles = df_merged['title']

In [3]:
app = flask.Flask(__name__)

In [4]:
def get_recommendations(first_article, model, vectorizer, training_vectors,title, ind):

    new_vec = model.transform(
        vectorizer.transform([first_article]))

    nn = NearestNeighbors(n_neighbors=10, metric='cosine', algorithm='brute')
    nn.fit(training_vectors)
    results = nn.kneighbors(new_vec)
    recommend_list = results[1][0]
    scores = results[0]

    rec_dict = defaultdict(list)

    ss = np.array(scores).flat
    for i, resp in enumerate(recommend_list):
        #print('--- ID ---\n', + resp)
        #print('--- dist ---\n', + ss[i])
        rec_dict["0"].append(df_merged.iloc[resp,1])
#         rec_dict['title'] = df_merged[title]
    
    rec_dict["0"].append(title)
    rec_dict["0"].append(topic_names.iloc[ind,0])
    return rec_dict

In [5]:
@app.route("/")
def visualize_page():

    
    with open("ted_html.html", 'r') as viz_file:
        return viz_file.read()

In [ ]:
@app.route("/ted", methods=["POST"])
def ted():
    """
    When A POST request with json data is made to this uri,
    Read the example from the json, predict probability and
    send it with a response
    """
    # Get decision score for our example that came with the request
    data = flask.request.json
    print(data)
    X = data#["example"])
    #X = str(X[0])
    title, score, talk_ind = process.extractOne(X, titles, scorer=fuzz.token_set_ratio)
    recs = get_recommendations(cleaned_talks[talk_ind],lda_mod, vect_mod, lda_data, title,talk_ind)

    print(recs.values())
    # Put the result in a nice dict so we can send it as json
    #results = {"recommends": recs[1]}
    #print(results)
    return flask.jsonify(recs['0'])
    #return flask.json.dump(recs,fp)
#--------- RUN WEB APP SERVER ------------#

# Start the app server on port 80
# (The default website port)
app.run(host='0.0.0.0')
app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2019 14:50:53] "GET / HTTP/1.1" 200 -
/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 14:51:00] "POST /ted HTTP/1.1" 200 -


climate
dict_values([['With the same humor and humanity he exuded in "An Inconvenient Truth," Al Gore spells out 15 ways that individuals can address climate change immediately, from buying a hybrid to inventing a new, hotter brand name for global warming.', "What's a proven way to lower your energy costs? Would you believe: learning what your neighbor pays. Alex Laskey shows how a quirk of human behavior can make us all better, wiser energy users, with lower bills to prove it.", "New York City residents produce 11,000 tons of garbage every day. Every day! This astonishing statistic is just one of the reasons Robin Nagle started a research project with the city's Department of Sanitation. She walked the routes, operated mechanical brooms, even drove a garbage truck herself--all so she could answer a simple-sounding but complicated question: who cleans up after us?", 'At the Taste3 conference, chef Dan Barber tells the story of a small farm in Spain that has found a humane way to produc

/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 14:53:56] "POST /ted HTTP/1.1" 200 -


Do schools kill creativity
dict_values([['Sir Ken Robinson makes an entertaining and profoundly moving case for creating an education system that nurtures (rather than undermines) creativity.', "With profound simplicity, Coach John Wooden redefines success and urges us all to pursue the best in ourselves. In this inspiring talk he shares the advice he gave his players at UCLA, quotes poetry and remembers his father's wisdom.", 'How can you help kids get a good start? In this heartfelt and personal talk, Colin Powell, the former U.S. Secretary of State, asks parents, friends and relatives to support children, starting before they even get to primary school, through community and a strong sense of responsibility.', "Imagine a country where girls must sneak out to go to school, with deadly consequences if they get caught learning. This was Afghanistan under the Taliban, and traces of that danger remain today. 22-year-old Shabana Basij-Rasikh runs a school for girls in Afghanistan. She cel

127.0.0.1 - - [26/May/2019 14:57:06] "GET / HTTP/1.1" 200 -
/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:395: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
127.0.0.1 - - [26/May/2019 14:57:12] "POST /ted HTTP/1.1" 200 -


schools kill
dict_values([['Sir Ken Robinson makes an entertaining and profoundly moving case for creating an education system that nurtures (rather than undermines) creativity.', "With profound simplicity, Coach John Wooden redefines success and urges us all to pursue the best in ourselves. In this inspiring talk he shares the advice he gave his players at UCLA, quotes poetry and remembers his father's wisdom.", 'How can you help kids get a good start? In this heartfelt and personal talk, Colin Powell, the former U.S. Secretary of State, asks parents, friends and relatives to support children, starting before they even get to primary school, through community and a strong sense of responsibility.', "Imagine a country where girls must sneak out to go to school, with deadly consequences if they get caught learning. This was Afghanistan under the Taliban, and traces of that danger remain today. 22-year-old Shabana Basij-Rasikh runs a school for girls in Afghanistan. She celebrates the po